<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/Python_Data_Analytics_Library/12_%EB%B6%84%EC%84%9D%EC%98%88%EC%A0%9C7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.font_manager as fm
fm.FontProperties(fname = "맑은 고딕")
mpl.rcParams["axes.unicode_minus"] = False # 음수표시

import seaborn as sns

plt.style.use("fivethirtyeight")

import warnings
warnings.filterwarnings("ignore")


import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import requests
# from selenium import webdriver
import urllib.parse

from bs4 import BeautifulSoup

# 우리나라 인구 소멸 위기 지역 분석

- 65세 이상 노인 인구와 20 ~ 39세 여성 인구를 비교해서 젊은 여성 인구가 노인 인구의 절반에 미달할 경우
- <한국 고용정보원 - 한국의 지방소멸에 관한 7가지 분석>

### 1. 데이터 준비

- 통계청 사이트 : http://kosis.co.kr/index/index.do



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
pop = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/population_raw_data.xlsx', header = 1)
pop.fillna(method='pad', inplace=True)
pop.rename(columns = {'행정구역별(읍면동)(1)':'광역시도', 
                             '행정구역별(읍면동)(2)':'시도', 
                             '합계':'인구수',
                             '항목':'구분'}, inplace=True)


pop.loc[pop["구분"] == "총인구(명)", "구분"] = "합계"
pop.loc[pop["구분"] == "총인구_남자(명)", "구분"] = "남자"
pop.loc[pop["구분"] == "총인구_여자(명)", "구분"] = "여자"
pop = pop[(pop['시도'] != '소계')]


pop

,광역시도,시도,구분,인구수,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세이상
6,서울특별시,종로구,합계,154969,15242,14057,10753,10504,9846,12652,12323,12556,10726,7366,6101,5377,3461,1580,565,147,21
7,서울특별시,종로구,남자,74880,6707,7194,5443,5206,4811,6214,5969,6204,5338,3527,2718,2329,1416,555,182,45,6
8,서울특별시,종로구,여자,80089,8535,6863,5310,5298,5035,6438,6354,6352,5388,3839,3383,3048,2045,1025,383,102,15
9,서울특별시,중구,합계,130957,10762,12057,11034,10933,8810,9926,10291,10786,9870,6665,5286,4333,2692,1234,476,92,17
10,서울특별시,중구,남자,63406,4841,6006,5548,5607,4586,4963,5096,5306,4799,3200,2370,1835,1029,370,133,26,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,제주특별자치도,제주시,남자,246161,17040,17382,15117,19865,19496,22822,20986,18803,15246,9288,7488,5488,3368,1355,254,64,5
833,제주특별자치도,제주시,여자,241527,13977,14899,14047,19171,18720,22001,19427,18162,15221,9918,8481,7364,5965,4008,1434,620,70
834,제주특별자치도,서귀포시,합계,177360,8664,10334,9476,13035,13348,15827,14803,14756,13018,8943,8086,6720,4854,2649,728,274,46
835,제주특별자치도,서귀포시,남자,90437,4908,5759,5085,6768,7122,8655,7930,7628,6526,4304,3828,2859,1750,702,121,24,X


In [52]:
##### 95 ~ 99세, 100세 이상의 값들 중 숫자가 아닌 값이 있는지 확인

pop["95~99세"].str.extract("([^0-9])").value_counts()
pop["100세이상"].str.extract("([^0-9])").value_counts()

AttributeError: ignored

In [64]:
##### 95 ~ 99세, 100세 이상의 값들 중 X값을 0으로 대체하고 숫자형으로 변환

pop.loc[pop["95~99세"] == "X", "95~99세"] = 0
pop.loc[pop["100세이상"] == "X", "100세이상"] = 0

pop["95~99세"].unique()
pop["100세이상"].unique()

pop["95~99세"] = pd.to_numeric(pop["95~99세"])
pop["100세이상"] = pd.to_numeric(pop["100세이상"])

pop.info()

In [66]:
pop['20-39세'] = pop['20~24세'] + pop['25~29세'] + \
                        pop['30~34세'] + pop['35~39세']
    
pop['65세이상'] = pop['65~69세'] + pop['70~74세'] + \
                        pop['75~79세'] + pop['80~84세'] + \
                        pop['85~89세'] + pop['90~94세'] + \
                        pop['95~99세'] + pop['100세이상']

pop.head(10)

,광역시도,시도,구분,인구수,20~24세,25~29세,30~34세,35~39세,40~44세,45~49세,50~54세,55~59세,60~64세,65~69세,70~74세,75~79세,80~84세,85~89세,90~94세,95~99세,100세이상,95세~99세,20-39세,65세이상
6,서울특별시,종로구,합계,154969,15242,14057,10753,10504,9846,12652,12323,12556,10726,7366,6101,5377,3461,1580,565,147,21,NaN,50556,24618
7,서울특별시,종로구,남자,74880,6707,7194,5443,5206,4811,6214,5969,6204,5338,3527,2718,2329,1416,555,182,45,6,NaN,24550,10778
8,서울특별시,종로구,여자,80089,8535,6863,5310,5298,5035,6438,6354,6352,5388,3839,3383,3048,2045,1025,383,102,15,NaN,26006,13840
9,서울특별시,중구,합계,130957,10762,12057,11034,10933,8810,9926,10291,10786,9870,6665,5286,4333,2692,1234,476,92,17,NaN,44786,20795
10,서울특별시,중구,남자,63406,4841,6006,5548,5607,4586,4963,5096,5306,4799,3200,2370,1835,1029,370,133,26,5,NaN,22002,8968
11,서울특별시,중구,여자,67551,5921,6051,5486,5326,4224,4963,5195,5480,5071,3465,2916,2498,1663,864,343,66,12,NaN,22784,11827
12,서울특별시,용산구,합계,227181,15289,20431,19789,20216,16585,19022,17673,17272,15423,10344,8396,7180,4847,2141,725,194,22,NaN,75725,33849
13,서울특별시,용산구,남자,110934,6954,10232,10207,10431,8372,9498,8754,8430,7468,4838,3739,3052,1852,759,192,62,5,NaN,37824,14499
14,서울특별시,용산구,여자,116247,8335,10199,9582,9785,8213,9524,8919,8842,7955,5506,4657,4128,2995,1382,533,132,17,NaN,37901,19350
15,서울특별시,성동구,합계,299688,22964,26568,24647,26456,21640,24238,23687,24039,20801,13760,10870,8810,5273,2241,724,181,23,NaN,100635,41882


In [68]:
pop1 = pd.pivot_table(pop, 
                     index = ['광역시도', '시도'], 
                     columns = ['구분'],
                     values = ['인구수', '20-39세', '65세이상'])
pop1

20-39세                  65세이상                    인구수                
구분            남자      여자      합계     남자     여자      합계      남자      여자      합계
광역시도 시도                                                                       
강원도  강릉시   29173   23558   52731  17822  24250   42072  108707  107650  216357
     고성군    4351    2253    6604   3003   4207    7212   14505   12639   27144
     동해시   10046    7712   17758   7238   9586   16825   44086   42694   86780
     삼척시    9188    6579   15767   6394   8863   15258   34827   32383   67210
     속초시    9400    7662   17062   5981   8497   14478   39015   39628   78643
...          ...     ...     ...    ...    ...     ...     ...     ...     ...
충청북도 진천군   15009    9819   24828   5417   7423   12845   47307   40218   87525
     청원구   34800   28707   63507   9780  13461   23244  102990   97591  200581
     청주시  134191  116716  250907  44695  60464  105159  429372  421566  850938
     충주시   31487   23947   55434  16481  22484   38968  110758  105688  216446
     흥덕구   45042   39860   84902  11866  15657   27524  135116  131185  266301

[261 rows x 9 columns]

- 소멸비율이라는 컬럼에 인구소멸위기지역을 계산하기 위한 식을 적용한다
- 이 비율이 1보다 작으면 인구소멸위기지역으로 볼 수 있다.

In [69]:
pop1['소멸비율'] = pop1['20-39세','여자'] / (pop1['65세이상','합계'] / 2)
pop1.head()

20-39세                65세이상  ...     인구수                      소멸비율
구분           남자     여자     합계     남자  ...      남자      여자      합계          
광역시도 시도                               ...                                  
강원도  강릉시  29173  23558  52731  17822  ...  108707  107650  216357  1.119890
     고성군   4351   2253   6604   3003  ...   14505   12639   27144  0.624792
     동해시  10046   7712  17758   7238  ...   44086   42694   86780  0.916731
     삼척시   9188   6579  15767   6394  ...   34827   32383   67210  0.862367
     속초시   9400   7662  17062   5981  ...   39015   39628   78643  1.058433

[5 rows x 10 columns]

- 소멸위기지역인지를 boolean으로 지정해 둔다

In [70]:
pop1['소멸위기지역'] = pop1['소멸비율'] < 1.0
pop1.head()

20-39세                65세이상  ...     인구수              소멸비율 소멸위기지역
구분           남자     여자     합계     남자  ...      여자      합계                 
광역시도 시도                               ...                                 
강원도  강릉시  29173  23558  52731  17822  ...  107650  216357  1.119890  False
     고성군   4351   2253   6604   3003  ...   12639   27144  0.624792   True
     동해시  10046   7712  17758   7238  ...   42694   86780  0.916731   True
     삼척시   9188   6579  15767   6394  ...   32383   67210  0.862367   True
     속초시   9400   7662  17062   5981  ...   39628   78643  1.058433  False

[5 rows x 11 columns]

In [71]:
pop1[pop1['소멸위기지역']==True].index.get_level_values(1)

Index(['고성군', '동해시', '삼척시', '양양군', '영월군', '인제군', '정선군', '철원군', '태백시', '평창군',
       '홍천군', '화천군', '횡성군', '가평군', '양평군', '연천군', '거창군', '고성군', '남해군', '밀양시',
       '사천시', '산청군', '의령군', '창녕군', '하동군', '함안군', '함양군', '합천군', '고령군', '군위군',
       '김천시', '문경시', '봉화군', '상주시', '성주군', '안동시', '영덕군', '영양군', '영주시', '영천시',
       '예천군', '울릉군', '울진군', '의성군', '청도군', '청송군', '동구', '영도구', '중구', '강화군',
       '옹진군', '강진군', '고흥군', '곡성군', '구례군', '담양군', '보성군', '신안군', '영광군', '영암군',
       '완도군', '장성군', '장흥군', '진도군', '함평군', '해남군', '화순군', '고창군', '김제시', '남원시',
       '무주군', '부안군', '순창군', '임실군', '장수군', '정읍시', '진안군', '공주시', '금산군', '논산시',
       '보령시', '부여군', '서천군', '예산군', '청양군', '태안군', '홍성군', '괴산군', '단양군', '보은군',
       '영동군', '옥천군'],
      dtype='object', name='시도')

- pivot_table로 잘 정리가 된 상태에서 .reset_index로 pivot_table의 result 속성을 다시 설정한다.

In [72]:
pop1.reset_index(inplace=True) 
pop1.head()

광역시도   시도 20-39세                ...     인구수                      소멸비율 소멸위기지역
구분               남자     여자     합계  ...      남자      여자      합계                 
0   강원도  강릉시  29173  23558  52731  ...  108707  107650  216357  1.119890  False
1   강원도  고성군   4351   2253   6604  ...   14505   12639   27144  0.624792   True
2   강원도  동해시  10046   7712  17758  ...   44086   42694   86780  0.916731   True
3   강원도  삼척시   9188   6579  15767  ...   34827   32383   67210  0.862367   True
4   강원도  속초시   9400   7662  17062  ...   39015   39628   78643  1.058433  False

[5 rows x 13 columns]

In [73]:
tmp_coloumns = [pop1.columns.get_level_values(0)[n] + \
                pop1.columns.get_level_values(1)[n] 
                for n in range(0,len(pop1.columns.get_level_values(0)))]

pop1.columns = tmp_coloumns

pop1.head()

,광역시도,시도,20-39세남자,20-39세여자,20-39세합계,65세이상남자,65세이상여자,65세이상합계,인구수남자,인구수여자,인구수합계,소멸비율,소멸위기지역
0,강원도,강릉시,29173,23558,52731,17822,24250,42072,108707,107650,216357,1.119890,False
1,강원도,고성군,4351,2253,6604,3003,4207,7212,14505,12639,27144,0.624792,True
2,강원도,동해시,10046,7712,17758,7238,9586,16825,44086,42694,86780,0.916731,True
3,강원도,삼척시,9188,6579,15767,6394,8863,15258,34827,32383,67210,0.862367,True
4,강원도,속초시,9400,7662,17062,5981,8497,14478,39015,39628,78643,1.058433,False


In [94]:
pop1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   광역시도      261 non-null    object 
 1   시도        261 non-null    object 
 2   20-39세여자  261 non-null    int64  
 3   20-39세합계  261 non-null    int64  
 4   65세이상합계   261 non-null    int64  
 5   인구수남자     261 non-null    int64  
 6   인구수여자     261 non-null    int64  
 7   인구수합계     261 non-null    int64  
 8   소멸비율      261 non-null    float64
 9   소멸위기지역    261 non-null    bool   
 10  ID        261 non-null    object 
dtypes: bool(1), float64(1), int64(6), object(3)
memory usage: 20.8+ KB


### 3. 지도 시각화를 위해 지역별 고유 ID 만들기

In [76]:
pop1['시도'].unique()

array(['강릉시', '고성군', '동해시', '삼척시', '속초시', '양구군', '양양군', '영월군', '원주시',
       '인제군', '정선군', '철원군', '춘천시', '태백시', '평창군', '홍천군', '화천군', '횡성군',
       '가평군', '고양시', '과천시', '광명시', '광주시', '구리시', '군포시', '권선구', '기흥구',
       '김포시', '남양주시', '단원구', '덕양구', '동두천시', '동안구', '만안구', '부천시', '분당구',
       '상록구', '성남시', '수원시', '수정구', '수지구', '시흥시', '안산시', '안성시', '안양시',
       '양주시', '양평군', '여주시', '연천군', '영통구', '오산시', '용인시', '의왕시', '의정부시',
       '이천시', '일산동구', '일산서구', '장안구', '중원구', '처인구', '파주시', '팔달구', '평택시',
       '포천시', '하남시', '화성시', '거제시', '거창군', '김해시', '남해군', '마산합포구', '마산회원구',
       '밀양시', '사천시', '산청군', '성산구', '양산시', '의령군', '의창구', '진주시', '진해구',
       '창녕군', '창원시', '통영시', '하동군', '함안군', '함양군', '합천군', '경산시', '경주시',
       '고령군', '구미시', '군위군', '김천시', '남구', '문경시', '봉화군', '북구', '상주시', '성주군',
       '안동시', '영덕군', '영양군', '영주시', '영천시', '예천군', '울릉군', '울진군', '의성군',
       '청도군', '청송군', '칠곡군', '포항시', '광산구', '동구', '서구', '달서구', '달성군', '수성구',
       '중구', '대덕구', '유성구', '강서구', '금정구', '기장군', '동래구', '부산진구', '사상구',
 

In [79]:
si_name = [None] * len(pop1)

tmp_gu_dict = {'수원':['장안구', '권선구', '팔달구', '영통구'], 
                       '성남':['수정구', '중원구', '분당구'], 
                       '안양':['만안구', '동안구'], 
                       '안산':['상록구', '단원구'], 
                       '고양':['덕양구', '일산동구', '일산서구'], 
                       '용인':['처인구', '기흥구', '수지구'], 
                       '청주':['상당구', '서원구', '흥덕구', '청원구'], 
                       '천안':['동남구', '서북구'], 
                       '전주':['완산구', '덕진구'], 
                       '포항':['남구', '북구'], 
                       '창원':['의창구', '성산구', '진해구', '마산합포구', '마산회원구'], 
                       '부천':['오정구', '원미구', '소사구']}

In [80]:
for n in pop1.index:
    if pop1['광역시도'][n][-3:] not in ['광역시', '특별시', '자치시']:
        if pop1['시도'][n][:-1]=='고성' and pop1['광역시도'][n]=='강원도':
            si_name[n] = '고성(강원)'
        elif pop1['시도'][n][:-1]=='고성' and pop1['광역시도'][n]=='경상남도':
            si_name[n] = '고성(경남)'
        else:
             si_name[n] = pop1['시도'][n][:-1]
                
        for keys, values in tmp_gu_dict.items():
            if pop1['시도'][n] in values:
                if len(pop1['시도'][n])==2:
                    si_name[n] = keys + ' ' + pop1['시도'][n]
                elif pop1['시도'][n] in ['마산합포구','마산회원구']:
                    si_name[n] = keys + ' ' + pop1['시도'][n][2:-1]
                else:
                    si_name[n] = keys + ' ' + pop1['시도'][n][:-1]
        
    elif pop1['광역시도'][n] == '세종특별자치시':
        si_name[n] = '세종'
        
    else:
        if len(pop1['시도'][n])==2:
            si_name[n] = pop1['광역시도'][n][:2] + ' ' + pop1['시도'][n]
        else:
            si_name[n] = pop1['광역시도'][n][:2] + ' ' + pop1['시도'][n][:-1]

In [82]:
si_name

['강릉',
 '고성(강원)',
 '동해',
 '삼척',
 '속초',
 '양구',
 '양양',
 '영월',
 '원주',
 '인제',
 '정선',
 '철원',
 '춘천',
 '태백',
 '평창',
 '홍천',
 '화천',
 '횡성',
 '가평',
 '고양',
 '과천',
 '광명',
 '광주',
 '구리',
 '군포',
 '수원 권선',
 '용인 기흥',
 '김포',
 '남양주',
 '안산 단원',
 '고양 덕양',
 '동두천',
 '안양 동안',
 '안양 만안',
 '부천',
 '성남 분당',
 '안산 상록',
 '성남',
 '수원',
 '성남 수정',
 '용인 수지',
 '시흥',
 '안산',
 '안성',
 '안양',
 '양주',
 '양평',
 '여주',
 '연천',
 '수원 영통',
 '오산',
 '용인',
 '의왕',
 '의정부',
 '이천',
 '고양 일산동',
 '고양 일산서',
 '수원 장안',
 '성남 중원',
 '용인 처인',
 '파주',
 '수원 팔달',
 '평택',
 '포천',
 '하남',
 '화성',
 '거제',
 '거창',
 '고성(경남)',
 '김해',
 '남해',
 '창원 합포',
 '창원 회원',
 '밀양',
 '사천',
 '산청',
 '창원 성산',
 '양산',
 '의령',
 '창원 의창',
 '진주',
 '창원 진해',
 '창녕',
 '창원',
 '통영',
 '하동',
 '함안',
 '함양',
 '합천',
 '경산',
 '경주',
 '고령',
 '구미',
 '군위',
 '김천',
 '포항 남구',
 '문경',
 '봉화',
 '포항 북구',
 '상주',
 '성주',
 '안동',
 '영덕',
 '영양',
 '영주',
 '영천',
 '예천',
 '울릉',
 '울진',
 '의성',
 '청도',
 '청송',
 '칠곡',
 '포항',
 '광주 광산',
 '광주 남구',
 '광주 동구',
 '광주 북구',
 '광주 서구',
 '대구 남구',
 '대구 달서',
 '대구 달성',
 '대구 동구',
 '대구 북구',
 '대구 서구',
 '대구 수성

In [83]:
pop1["ID"] = si_name

In [85]:
del pop1['20-39세남자']
del pop1['65세이상남자']
del pop1['65세이상여자']

pop1.head()

,광역시도,시도,20-39세여자,20-39세합계,65세이상합계,인구수남자,인구수여자,인구수합계,소멸비율,소멸위기지역,ID
0,강원도,강릉시,23558,52731,42072,108707,107650,216357,1.119890,False,강릉
1,강원도,고성군,2253,6604,7212,14505,12639,27144,0.624792,True,고성(강원)
2,강원도,동해시,7712,17758,16825,44086,42694,86780,0.916731,True,동해
3,강원도,삼척시,6579,15767,15258,34827,32383,67210,0.862367,True,삼척
4,강원도,속초시,7662,17062,14478,39015,39628,78643,1.058433,False,속초


### 4. 지도 시각화

In [88]:
import folium
import json

In [ ]:
pop = ('/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/skorea_municipalities_geo_simple_update.json')

In [104]:
pop2 = pop1
pop2.set_index(pop2['ID'], inplace = True)

In [105]:
pop2

,광역시도,시도,20-39세여자,20-39세합계,65세이상합계,인구수남자,인구수여자,인구수합계,소멸비율,소멸위기지역,ID
ID,,,,,,,,,,,
강릉,강원도,강릉시,23558,52731,42072,108707,107650,216357,1.119890,False,강릉
고성(강원),강원도,고성군,2253,6604,7212,14505,12639,27144,0.624792,True,고성(강원)
동해,강원도,동해시,7712,17758,16825,44086,42694,86780,0.916731,True,동해
삼척,강원도,삼척시,6579,15767,15258,34827,32383,67210,0.862367,True,삼척
속초,강원도,속초시,7662,17062,14478,39015,39628,78643,1.058433,False,속초
...,...,...,...,...,...,...,...,...,...,...,...
진천,충청북도,진천군,9819,24828,12845,47307,40218,87525,1.528844,False,진천
청주 청원,충청북도,청원구,28707,63507,23244,102990,97591,200581,2.470057,False,청주 청원
청주,충청북도,청주시,116716,250907,105159,429372,421566,850938,2.219800,False,청주


In [106]:
geo_path = '/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/skorea_municipalities_geo_simple_update.json'
geo_data = json.load(open(geo_path, encoding='utf-8'))
map =folium.Map(location=[36.2002, 127.054], zoom_start=7)
map.choropleth(geo_data,
              data= pop2['인구수합계'],
              columns = [pop2.index, pop2["인구수합계"]],
              fill_color='YlGnBu',
              key_on='feature.id')
map